In [28]:
# reporting/quarterly_report/modules/granting.py
from __future__ import annotations

import logging, sqlite3, datetime
from pathlib import Path
from typing import List
import calendar
import pandas as pd
from datetime import date
from typing import List, Tuple,Union
import numpy as np
from great_tables import GT, loc, style, html
import re

# our project
from ingestion.db_utils import (
    init_db,                                 # create tables if missing
    fetch_latest_table_data,                 # new version!
    get_alias_last_load,
    get_variable_status, 
    load_report_params                   # to inspect results
)

from reporting.quarterly_report.utils import RenderContext, BaseModule
from reporting.quarterly_report.report_utils.granting_utils import enrich_grants, _ensure_timedelta_cols, _coerce_date_columns
from reporting.quarterly_report.utils import Database, RenderContext


import altair as alt
from altair_saver import save
import selenium.webdriver



# ─────────────────────────────────────────────────────────────
# 2) open DB – change path if you work on a copy
# ─────────────────────────────────────────────────────────────
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")

init_db(db_path=DB_PATH)            # no-op if tables already exist

db = Database(str(DB_PATH))         # thin sqlite3 wrapper
conn = db.conn
report = 'Quarterly_Report'

CALLS_TYPES_LIST = ['STG', 'ADG', 'POC', 'COG', 'SYG', 'StG', 'CoG', 'AdG', 'SyG', 'PoC', 'CSA']
# ──────────────────────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────────────────────

def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year



def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)



def months_in_scope(cutoff: pd.Timestamp) -> list[str]:
    """
    Returns list of month names from January to last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).strftime("%B").tolist()

    return months

def determine_po_category(row):

    instrument = str(row.get('Instrument', '')).strip()
    topic = str(row.get('Topic', '')).strip()

    try:
        if topic and any(call_type in topic for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in topic).upper()
            return category
        elif instrument and any(call_type in instrument for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in instrument).upper()
            return category
        return ''
    except Exception as e:
        raise

def determine_po_category_po_list(row):

    summa = str(row.get('PO Purchase Order Item Desc', '')).strip()
    abac = str(row.get('PO ABAC SAP Reference', '')).strip()

    try:
        if summa and any(call_type in summa for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in summa).upper()
            return category
        elif abac and any(call_type in abac for call_type in CALLS_TYPES_LIST):
            category = next(call_type for call_type in CALLS_TYPES_LIST if call_type in abac).upper()
            return category
        return ''
    except Exception as e:
        raise

def extract_project_number(row):
    """
    Extract project number from 'Inv Text' if 'v_check_payment_type' contains RP patterns,
    otherwise return original 'v_check_payment_type' value
    """
    payment_type = row['v_check_payment_type']
    inv_text = row['Inv Text']
    
    # Handle NaN values
    if pd.isna(payment_type):
        return payment_type
    
    # Convert to string to handle any data type
    payment_type_str = str(payment_type)
    
    # Check if the payment_type contains RP patterns:
    # - Original pattern: RP + number + = + FP/IP (e.g., RP4=FP, RP2=IP)
    # - New pattern: RP + number + - + FP/IP (e.g., RP4-FP, RP2-IP)
    rp_patterns = [
        r'RP\d+=(?:FP|IP)',  # Original pattern: RP4=FP, RP2=IP, etc.
        r'RP\d+-(?:FP|IP)'   # New pattern: RP4-FP, RP2-IP, etc.
    ]
    
    # Check if any of the RP patterns match
    has_rp_pattern = any(re.search(pattern, payment_type_str) for pattern in rp_patterns)
    
    if has_rp_pattern:
        # Extract the numerical part from Inv Text column
        if pd.notna(inv_text):
            inv_text_str = str(inv_text).strip()
            # Extract leading digits from Inv Text
            number_match = re.match(r'^(\d+)', inv_text_str)
            if number_match:
                return number_match.group(1)
        
        # If no number found in Inv Text, return original payment_type
        return payment_type
    
    # Return original v_check_payment_type if no RP pattern found
    return payment_type


def map_project_to_call_type(project_num, mapping_dict):
    # If it's a numeric string, try to convert and lookup
    try:
        # Try to convert to int for lookup
        numeric_key = int(project_num)
        if numeric_key in mapping_dict:
            return mapping_dict[numeric_key]
    except (ValueError, TypeError):
        # If conversion fails, it's a non-numeric string like 'EXPERTS'
        pass
    
    # Return original value if no match found
    return project_num

def map_call_type_with_experts(row, grant_map):
    """
    Map call_type based on project_number and Inv Parking Person Id
    """
    project_num = row['project_number']
    contract_type = row['v_payment_type']
    
    # First, try to map using grant_map (convert project_num to int if possible)
    try:
        numeric_key = int(project_num)
        if numeric_key in grant_map:
            return grant_map[numeric_key]
    except (ValueError, TypeError):
        pass
    
    # If project_number is 'EXPERTS', keep it as 'EXPERTS'
    if str(project_num).upper() == 'EXPERTS' or str(contract_type).upper() == 'EXPERTS':
        return 'EXPERTS'
    
    # Return original project_number if no conditions are met
    return project_num

def map_payment_type(row):
    if row['v_payment_type'] == 'Other' and row['Pay Workflow Last AOS Person Id'] == 'WALASOU':
        return 'EXPERTS'
    return row['v_payment_type']

# Instead, handle conversion in the mapping function
def safe_map_project_to_call_type(project_num, mapping_dict):
    """
    Maps project number to call type, handles all data type issues internally
    """
    try:
        # Handle NaN values
        if pd.isna(project_num):
            return None
            
        # Convert whatever format to integer for lookup
        if isinstance(project_num, str):
            # Handle strings like '4500053782.0'
            if project_num.endswith('.0'):
                numeric_key = int(project_num[:-2])
            else:
                numeric_key = int(float(project_num))
        else:
            # Handle numeric values (float/int)
            numeric_key = int(float(project_num))
            
        # Lookup in mapping dictionary
        if numeric_key in mapping_dict:
            result = mapping_dict[numeric_key]
            if pd.notna(result) and result != '':
                return result
                
    except (ValueError, TypeError, OverflowError):
        # Any conversion error, return None
        pass
    
    return None

# Apply mapping without converting the whole column
def apply_conditional_mapping(row):
    current_call_type = row['call_type']
    po_key = row['PO Purchase Order Key']  # Use as-is, no conversion
    
    should_map = (
        pd.isna(current_call_type) or 
        current_call_type == '' or 
        current_call_type not in CALLS_TYPES_LIST or 
        current_call_type in ['EXPERTS', 'CSA']
    )
    
    if should_map:
        mapped_value = safe_map_project_to_call_type(po_key, po_map)
        return mapped_value if mapped_value is not None else current_call_type
    else:
        return current_call_type



In [29]:
PAYMENTS_ALIAS = "payments_summa"
CALLS_ALIAS = 'call_overview'
PAYMENTS_TIMES_ALIAS = 'payments_summa_time'
PO_ALIAS = 'c0_po_summa'

cutoff = pd.to_datetime("2025-04-15")
report_params = load_report_params(report_name=report, db_path=db_path)


table_colors = report_params.get('TABLE_COLORS', {})
BLUE = table_colors.get("BLUE", "#004A99")
LIGHT_BLUE = table_colors.get("LIGHT_BLUE", "#d6e6f4")
DARK_BLUE = table_colors.get("DARK_BLUE", "#01244B")
SUB_TOTAL_BACKGROUND = table_colors.get("subtotal_background_color", "#E6E6FA")

df_paym = fetch_latest_table_data(conn, PAYMENTS_ALIAS, cutoff)
df_paym_times = fetch_latest_table_data(conn, PAYMENTS_TIMES_ALIAS, cutoff)
df_calls =  fetch_latest_table_data(conn, CALLS_ALIAS , cutoff)
df_po = fetch_latest_table_data(conn, PO_ALIAS, cutoff)

DEBUG:root:Fetching latest data for table_alias: payments_summa, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for payments_summa: [('2025-06-03T06:26:12.795557', 1)]
DEBUG:root:Checking upload_id: 1, uploaded_at: 2025-06-03T06:26:12.795557
DEBUG:root:Fetched 6391 rows from payments_summa with upload_id 1
DEBUG:root:Fetching latest data for table_alias: payments_summa_time, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for payments_summa_time: [('2025-06-03T06:26:38.788420', 2)]
DEBUG:root:Checking upload_id: 2, uploaded_at: 2025-06-03T06:26:38.788420
DEBUG:root:Fetched 4992 rows from payments_summa_time with upload_id 2
DEBUG:root:Fetching latest data for table_alias: call_overview, cutoff: 2025-04-15T00:00:00
DEBUG:root:Upload log query results for call_overview: [('2025-06-03T06:28:40.771822', 3)]
DEBUG:root:Checking upload_id: 3, uploaded_at: 2025-06-03T06:28:40.771822
DEBUG:root:Fetched 13295 rows from call_overview with upload_id 3
DEBUG:root:F

In [30]:
df_paym['v_payment_type'] = df_paym.apply(map_payment_type, axis=1)
# Filter the dataframe
df_paym = df_paym[df_paym['Pay Document Type Desc'].isin(['Payment Directive', 'Exp Pre-financing'])]
# Keep all rows where v_payment_type is not 'Other'
df_paym = df_paym[df_paym['v_payment_type'] != 'Other']
df_paym = df_paym[df_paym['Pay Payment Key'].notnull()]

df_paym['project_number'] = df_paym.apply(extract_project_number, axis=1)

# Assuming your DataFrame is called 'df'
df_calls['CALL_TYPE'] = df_calls.apply(determine_po_category, axis=1)
grant_map = df_calls.set_index('Grant Number')['CALL_TYPE'].to_dict()

#PO ORDERS MAP
df_po['CALL_TYPE']  = df_po.apply(determine_po_category_po_list, axis=1)

po_map = df_po[
    df_po['CALL_TYPE'].notna() & 
    (df_po['CALL_TYPE'].str.strip() != '')
].set_index('PO Purchase Order Key')['CALL_TYPE'].to_dict()

# Apply the mapping
df_paym['call_type'] = df_paym['project_number'].apply(lambda x: map_project_to_call_type(x, grant_map))
df_paym['call_type'] = df_paym.apply(lambda row: map_call_type_with_experts(row, grant_map), axis=1)


# Clean call_type column only (not PO keys)
df_paym['call_type'] = df_paym['call_type'].astype(str).str.strip().replace(['nan', ''], np.nan)
# Apply the mapping
df_paym['call_type'] = df_paym.apply(apply_conditional_mapping, axis=1)
# This preserves NaN values as NaN instead of causing errors
df_paym['PO Purchase Order Key'] = pd.to_numeric(df_paym['PO Purchase Order Key'], errors='coerce').astype('Int64')

df_paym['Pay Workflow Last AOS Action Date'] = pd.to_datetime(
    df_paym['Pay Workflow Last AOS Action Date'], 
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

quarter_dates = get_scope_start_end(cutoff=cutoff)
last_valid_date = quarter_dates[1]

df_paym = df_paym[
    df_paym['Pay Workflow Last AOS Action Date'] <= last_valid_date
].copy()

df_paym = df_paym[df_paym['call_type'] != 'CSA']

In [31]:
# Step 1: Create v_payment_in_time column in df_paym_times
print("=== STEP 1: Creating v_payment_in_time column ===")

# Convert Pay Delay Late Payment Flag (Y/N) to 1/0
df_paym_times['v_payment_in_time'] = df_paym_times['Pay Delay Late Payment Flag (Y/N)'].apply(
    lambda x: 1 if x == 'N' else 0
)

print("v_payment_in_time value counts:")
print(df_paym_times['v_payment_in_time'].value_counts())
print("\nOriginal flag vs new column:")
print(df_paym_times[['Pay Delay Late Payment Flag (Y/N)', 'v_payment_in_time']].value_counts())

# Step 2: Clean Pay Payment Key and create mappings
print("\n=== STEP 2: Creating payment mappings ===")

# Filter df_paym_times to only include rows we need for mapping
df_times_clean = df_paym_times.dropna(subset=['Pay Payment Key']).copy()

# Clean Pay Payment Key for mapping (convert to integers)
def safe_convert_to_int(value):
    """Safely convert payment key to integer"""
    try:
        if pd.isna(value):
            return None
        if isinstance(value, str):
            # Handle strings like '2551003294.0'
            if value.endswith('.0'):
                return int(value[:-2])
            else:
                return int(float(value))
        else:
            return int(float(value))
    except (ValueError, TypeError, OverflowError):
        return None

# Convert Pay Payment Key to integers for mapping (keep all rows)
df_times_clean['Pay_Payment_Key_Int'] = df_times_clean['Pay Payment Key'].apply(safe_convert_to_int)

# Count conversion issues but don't drop rows
conversion_failed = df_times_clean['Pay_Payment_Key_Int'].isna().sum()
conversion_success = df_times_clean['Pay_Payment_Key_Int'].notna().sum()

print(f"Payment key conversions - Success: {conversion_success}, Failed: {conversion_failed}")

# Create mappings only from successfully converted keys (but keep all rows in dataframe)
valid_conversions = df_times_clean['Pay_Payment_Key_Int'].notna()
mapping_data = df_times_clean[valid_conversions].copy()
mapping_data['Pay_Payment_Key_Int'] = mapping_data['Pay_Payment_Key_Int'].astype(int)

print(f"Payment times data: {len(df_times_clean)} total rows, {len(mapping_data)} usable for mapping")

# Create the three mappings (only from rows with valid conversions)
payment_key_to_ttp_gross = mapping_data.set_index('Pay_Payment_Key_Int')['Pay Delay With Suspension'].to_dict()
payment_key_to_ttp_net = mapping_data.set_index('Pay_Payment_Key_Int')['Pay Delay Without Suspension'].to_dict()
payment_key_to_payment_in_time = mapping_data.set_index('Pay_Payment_Key_Int')['v_payment_in_time'].to_dict()

print(f"TTP Gross mapping created: {len(payment_key_to_ttp_gross)} entries")
print(f"TTP Net mapping created: {len(payment_key_to_ttp_net)} entries")
print(f"Payment in time mapping created: {len(payment_key_to_payment_in_time)} entries")

# Step 3: Split dataframe and apply mappings selectively
print("\n=== STEP 3: Split, Map, and Merge Strategy ===")

# Check conditions in df_paym
exp_prefi_mask = df_paym['Pay Document Type Desc'] == 'Exp Pre-financing'
payment_directive_mask = df_paym['Pay Document Type Desc'] == 'Payment Directive'
other_mask = ~(exp_prefi_mask | payment_directive_mask)

print(f"Rows with 'Exp Pre-financing': {exp_prefi_mask.sum()}")
print(f"Rows with 'Payment Directive': {payment_directive_mask.sum()}")
print(f"Other document types: {other_mask.sum()}")
print(f"Total rows in df_paym: {len(df_paym)}")

# Step 3.1: Split the dataframe
df_exp_prefi = df_paym[exp_prefi_mask].copy()
df_payment_directive = df_paym[payment_directive_mask].copy()
df_other = df_paym[other_mask].copy()

print(f"\nDataframes split:")
print(f"- Exp Pre-financing: {len(df_exp_prefi)} rows")
print(f"- Payment Directive: {len(df_payment_directive)} rows")
print(f"- Other types: {len(df_other)} rows")

# Step 3.2: Apply mapping ONLY to Exp Pre-financing dataframe
if len(df_exp_prefi) > 0:
    print("\nApplying mappings to Exp Pre-financing dataframe...")
    
    # Mapping function for payment data
    def map_payment_data(pay_key, mapping_dict):
        """Map payment key to corresponding value"""
        try:
            if pd.isna(pay_key):
                return np.nan
                
            # Convert pay_key to int for lookup
            if isinstance(pay_key, str):
                if pay_key.endswith('.0'):
                    numeric_key = int(pay_key[:-2])
                else:
                    numeric_key = int(float(pay_key))
            else:
                numeric_key = int(float(pay_key))
                
            # Lookup in mapping
            if numeric_key in mapping_dict:
                return mapping_dict[numeric_key]
            else:
                return np.nan
                
        except (ValueError, TypeError, OverflowError):
            return np.nan
    
    # Add the three new columns to Exp Pre-financing dataframe ONLY
    df_exp_prefi['v_TTP_GROSS'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_ttp_gross)
    )
    
    df_exp_prefi['v_TTP_NET'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_ttp_net)
    )
    
    df_exp_prefi['v_payment_in_time'] = df_exp_prefi['Pay Payment Key'].apply(
        lambda x: map_payment_data(x, payment_key_to_payment_in_time)
    )
    
    print("Mapping applied to Exp Pre-financing rows!")
    
    # Show sample of what was mapped
    sample_mapped = df_exp_prefi[['Pay Payment Key', 'Pay Document Type Desc', 'v_TTP_GROSS', 'v_TTP_NET', 'v_payment_in_time']].head()
    print("Sample mapped rows:")
    print(sample_mapped)
else:
    print("No Exp Pre-financing rows to map!")

# Step 3.3: Payment Directive dataframe - ADD columns but DON'T change any existing values
if len(df_payment_directive) > 0:
    print("\nPreserving Payment Directive dataframe completely...")
    
    # Check if these columns already exist in Payment Directive rows
    if 'v_TTP_GROSS' in df_payment_directive.columns:
        print("v_TTP_GROSS already exists in Payment Directive - preserving original values")
    else:
        # Only add columns if they don't exist, with NaN values
        df_payment_directive['v_TTP_GROSS'] = np.nan
        
    if 'v_TTP_NET' in df_payment_directive.columns:
        print("v_TTP_NET already exists in Payment Directive - preserving original values")
    else:
        df_payment_directive['v_TTP_NET'] = np.nan
        
    if 'v_payment_in_time' in df_payment_directive.columns:
        print("v_payment_in_time already exists in Payment Directive - preserving original values")
    else:
        df_payment_directive['v_payment_in_time'] = np.nan
    
    print("Payment Directive rows completely preserved!")

# Step 3.4: Handle other document types
if len(df_other) > 0:
    print("Adding columns to other document types with NaN values...")
    df_other['v_TTP_GROSS'] = np.nan
    df_other['v_TTP_NET'] = np.nan
    df_other['v_payment_in_time'] = np.nan

# Step 3.5: Merge all dataframes back together
print("\nMerging dataframes back together...")

dataframes_to_merge = []
if len(df_exp_prefi) > 0:
    dataframes_to_merge.append(df_exp_prefi)
if len(df_payment_directive) > 0:
    dataframes_to_merge.append(df_payment_directive)
if len(df_other) > 0:
    dataframes_to_merge.append(df_other)

if dataframes_to_merge:
    df_paym = pd.concat(dataframes_to_merge, ignore_index=True)
    print(f"Dataframes merged successfully! Final shape: {df_paym.shape}")
else:
    print("Warning: No dataframes to merge!")

# Step 4: Check results
print("\n=== STEP 4: Results ===")
print("v_TTP_GROSS value counts:")
print(df_paym['v_TTP_GROSS'].value_counts(dropna=False))
print("\nv_TTP_NET value counts:")
print(df_paym['v_TTP_NET'].value_counts(dropna=False))
print("\nv_payment_in_time value counts:")
print(df_paym['v_payment_in_time'].value_counts(dropna=False))

# Show sample of mapped rows
print("\nSample of mapped rows:")
mapped_sample = df_paym[exp_prefi_mask & df_paym['v_TTP_GROSS'].notna()][
    ['Pay Payment Key', 'Pay Document Type Desc', 'v_TTP_GROSS', 'v_TTP_NET', 'v_payment_in_time']
].head()
print(mapped_sample)

=== STEP 1: Creating v_payment_in_time column ===
v_payment_in_time value counts:
v_payment_in_time
1    4776
0     216
Name: count, dtype: int64

Original flag vs new column:
Pay Delay Late Payment Flag (Y/N)  v_payment_in_time
N                                  1                    4776
Y                                  0                     216
Name: count, dtype: int64

=== STEP 2: Creating payment mappings ===
Payment key conversions - Success: 4992, Failed: 0
Payment times data: 4992 total rows, 4992 usable for mapping
TTP Gross mapping created: 4992 entries
TTP Net mapping created: 4992 entries
Payment in time mapping created: 4992 entries

=== STEP 3: Split, Map, and Merge Strategy ===
Rows with 'Exp Pre-financing': 347
Rows with 'Payment Directive': 2159
Other document types: 0
Total rows in df_paym: 2506

Dataframes split:
- Exp Pre-financing: 347 rows
- Payment Directive: 2159 rows
- Other types: 0 rows

Applying mappings to Exp Pre-financing dataframe...
Mapping applied to

/tmp/ipykernel_128910/632175654.py:194: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mapped_sample = df_paym[exp_prefi_mask & df_paym['v_TTP_GROSS'].notna()][


In [85]:
df_paym.to_excel('paym.xlsx')

In [32]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple

# Existing periodicity functions
def determine_epoch_year(cutoff_date: pd.Timestamp) -> int:
    """
    Returns the correct reporting year.
    If the cutoff is in January, then we are reporting for the *previous* year.
    """
    return cutoff_date.year - 1 if cutoff_date.month == 1 else cutoff_date.year

def get_scope_start_end(cutoff: pd.Timestamp) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Unified scope logic with year transition:
    • If cutoff is in January → report full previous year
    • Otherwise → return start of year to quarter-end
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        return pd.Timestamp(year=year, month=1, day=1), pd.Timestamp(year=year, month=12, day=31)

    def quarter_end(cutoff: pd.Timestamp) -> pd.Timestamp:
        first_day = cutoff.replace(day=1)
        last_month = first_day - pd.offsets.MonthBegin()
        m = last_month.month

        if m <= 3:
            return pd.Timestamp(year=cutoff.year, month=3, day=31)
        elif m <= 6:
            return pd.Timestamp(year=cutoff.year, month=6, day=30)
        elif m <= 9:
            return pd.Timestamp(year=cutoff.year, month=9, day=30)
        else:
            return pd.Timestamp(year=cutoff.year, month=12, day=31)

    return pd.Timestamp(year=cutoff.year, month=1, day=1), quarter_end(cutoff)

def months_in_scope(cutoff: pd.Timestamp) -> list[str]:
    """
    Returns list of month names from January to last *full* month before cutoff.
    Handles year rollover if cutoff is in January.
    """
    if cutoff.month == 1:
        year = cutoff.year - 1
        end_month = 12
    else:
        year = cutoff.year
        end_month = cutoff.month - 1

    months = pd.date_range(
        start=pd.Timestamp(year=year, month=1, day=1),
        end=pd.Timestamp(year=year, month=end_month, day=1),
        freq="MS"
    ).strftime("%B").tolist()

    return months

def create_quarterly_payment_tables(df_paym, cutoff_date=None):
    """
    Create quarterly payment tables matching the format from the Excel file
    - Amount summing: All v_amount_to_sum per payment key, regrouped by fund source
    - Number of payments: Count unique Pay Payment Key occurrences (deduplicated)
    - Assumes df_paym is already filtered for the correct time scope
    """
    
    print("=== QUARTERLY PAYMENT TABLES GENERATION ===")
    
    # Step 1: Set cutoff date for metadata
    if cutoff_date is None:
        cutoff_date = pd.Timestamp.now()
    elif isinstance(cutoff_date, str):
        cutoff_date = pd.Timestamp(cutoff_date)
    
    print(f"Cutoff date: {cutoff_date}")
    
    # Step 2: Get reporting metadata (for reference)
    reporting_year = determine_epoch_year(cutoff_date)
    scope_start, scope_end = get_scope_start_end(cutoff_date)
    months_in_report = months_in_scope(cutoff_date)
    
    print(f"Reporting year: {reporting_year}")
    print(f"Expected scope: {scope_start} to {scope_end}")
    print(f"Note: Assuming df_paym is already filtered for this scope")
    
    # Step 3: Validate required columns
    required_columns = [
        'Pay Payment Key', 
        'v_amount_to_sum', 
        'Fund Source',
        'v_payment_type', 
        'Pay Workflow Last AOS Action Date',
        'Programme'
    ]
    
    # Check for optional call_type column
    optional_columns = ['call_type', 'Call Type', 'v_call_type']
    call_type_col = None
    for col in optional_columns:
        if col in df_paym.columns:
            call_type_col = col
            print(f"Found call type column: {col}")
            break
    
    if call_type_col:
        required_columns.append(call_type_col)
    else:
        print("No call_type column found - will use Fund Source only")
    
    missing_columns = [col for col in required_columns if col not in df_paym.columns]
    if missing_columns:
        print(f"ERROR: Missing required columns: {missing_columns}")
        return None
    
    print("✓ All required columns present")
    
    # Step 4: Create working dataframe (skip date filtering since already done)
    df_work = df_paym[required_columns].copy()
    
    # Check for any remaining invalid dates
    invalid_dates = df_work['Pay Workflow Last AOS Action Date'].isna().sum()
    if invalid_dates > 0:
        print(f"WARNING: {invalid_dates} rows with invalid dates found, removing them")
        df_work = df_work.dropna(subset=['Pay Workflow Last AOS Action Date'])
    
    print(f"Working dataset: {len(df_work)} rows")
    
    if len(df_work) == 0:
        print("ERROR: No data available after validation")
        return None
    
    # Create quarter and year columns
    df_work['Quarter'] = df_work['Pay Workflow Last AOS Action Date'].dt.to_period('Q')
    df_work['Year'] = df_work['Pay Workflow Last AOS Action Date'].dt.year
    df_work['Quarter_Label'] = df_work['Quarter'].astype(str)
    
    print(f"Actual date range: {df_work['Pay Workflow Last AOS Action Date'].min()} to {df_work['Pay Workflow Last AOS Action Date'].max()}")
    print(f"Quarters found: {sorted(df_work['Quarter_Label'].unique())}")
    
    # Step 5: Map payment types and fund sources
    payment_type_mapping = {
        'IP': 'Interim Payments',
        'FP': 'Final Payments', 
        'PF': 'Pre-financing',
        'EXPERTS': 'Experts and Support'
    }
    
    # Keep original fund sources for now (don't map to C1/E0 yet)
    df_work['Payment_Type_Desc'] = df_work['v_payment_type'].map(payment_type_mapping)
    
    # Use call_type if available, otherwise use Fund Source
    if call_type_col:
        df_work['Call_Type_Display'] = df_work[call_type_col]
        print(f"Call types found: {sorted(df_work['Call_Type_Display'].unique())}")
    else:
        df_work['Call_Type_Display'] = df_work['Fund Source']
        print(f"Using Fund Source as call type: {sorted(df_work['Call_Type_Display'].unique())}")
    
    # Handle unmapped payment types
    unmapped_payments = df_work[df_work['Payment_Type_Desc'].isna()]['v_payment_type'].unique()
    if len(unmapped_payments) > 0:
        print(f"WARNING: Unmapped payment types found: {unmapped_payments}")
        # Keep unmapped ones with their original value
        df_work['Payment_Type_Desc'] = df_work['Payment_Type_Desc'].fillna(df_work['v_payment_type'])
    
    # Step 6: Split by Programme (H2020 and HEU)
    programmes = df_work['Programme'].unique()
    print(f"Programmes found: {programmes}")
    
    results = {
        'metadata': {
            'cutoff_date': cutoff_date,
            'reporting_year': reporting_year,
            'scope_start': scope_start,
            'scope_end': scope_end,
            'months_in_scope': months_in_report,
            'actual_date_range': {
                'start': df_work['Pay Workflow Last AOS Action Date'].min(),
                'end': df_work['Pay Workflow Last AOS Action Date'].max()
            },
            'call_type_column': call_type_col,
            'has_call_types': call_type_col is not None
        },
        'tables': {}
    }
    
    for programme in programmes:
        if programme not in ['H2020', 'HEU']:
            print(f"Skipping programme: {programme}")
            continue
            
        print(f"\n=== Processing {programme} ===")
        df_prog = df_work[df_work['Programme'] == programme].copy()
        
        if len(df_prog) == 0:
            print(f"No data for {programme}")
            continue
        
        # Create aggregation tables
        tables = create_programme_tables(df_prog, programme, reporting_year)
        results['tables'][programme] = tables
    
    return results

def create_programme_tables(df_prog, programme_name, reporting_year):
    """
    Create all payment type tables for a specific programme
    """
    
    tables = {}
    
    # Get unique payment types in this programme
    payment_types = df_prog['Payment_Type_Desc'].dropna().unique()
    
    for payment_type in payment_types:
        print(f"  Creating table for: {payment_type}")
        
        df_type = df_prog[df_prog['Payment_Type_Desc'] == payment_type].copy()
        
        if len(df_type) == 0:
            continue
            
        # Create quarterly aggregation
        quarterly_table = create_quarterly_aggregation(df_type, payment_type, reporting_year)
        tables[payment_type] = quarterly_table
    
    # Create overall summary table
    print(f"  Creating overall summary table")
    overall_table = create_quarterly_aggregation(df_prog, "All Payments", reporting_year)
    tables['All_Payments'] = overall_table
    
    return tables

def create_quarterly_aggregation(df_type, payment_type_name, reporting_year):
    """
    Create quarterly aggregation table for a specific payment type
    - Amounts: Sum all v_amount_to_sum (including by call type/fund source)
    - Transactions: Count unique Pay Payment Key
    - VOBU/EFTA: Sum only EFTA and VOBU fund sources
    """
    
    # Create base aggregation structure
    agg_data = []
    
    # Get all quarters in the data
    quarters = sorted(df_type['Quarter'].unique())
    
    for quarter in quarters:
        df_q = df_type[df_type['Quarter'] == quarter].copy()
        
        # Get call types for this quarter (using Call_Type_Display)
        call_types = df_q['Call_Type_Display'].unique()
        
        quarter_row = {
            'Quarter': str(quarter),
            'Quarter_Short': f"{quarter.quarter}Q{quarter.year}",
            'Year': quarter.year,
            'Payment_Type': payment_type_name,
            'Reporting_Year': reporting_year
        }
        
        # AMOUNTS: Sum all v_amount_to_sum by call type
        total_amount_all_types = 0
        vobu_efta_amount_all_types = 0
        
        for call_type in call_types:
            df_call_type = df_q[df_q['Call_Type_Display'] == call_type]
            
            # Total amount for this call type
            total_amount = df_call_type['v_amount_to_sum'].sum()
            quarter_row[f'Total_Amount_{call_type}'] = total_amount
            total_amount_all_types += total_amount
            
            # VOBU/EFTA amount: Only sum EFTA and VOBU fund sources
            df_vobu_efta = df_call_type[df_call_type['Fund Source'].isin(['VOBU', 'EFTA'])]
            vobu_efta_amount = df_vobu_efta['v_amount_to_sum'].sum()
            quarter_row[f'VOBU_EFTA_Amount_{call_type}'] = vobu_efta_amount
            vobu_efta_amount_all_types += vobu_efta_amount
            
            # TRANSACTIONS: Count unique Pay Payment Key for this call type
            unique_transactions_call_type = df_call_type['Pay Payment Key'].nunique()
            quarter_row[f'No_of_Transactions_{call_type}'] = unique_transactions_call_type
        
        # TRANSACTIONS: Count unique Pay Payment Key (deduplicated across all call types)
        unique_transactions = df_q['Pay Payment Key'].nunique()
        quarter_row['No_of_Transactions'] = unique_transactions
        
        # OVERALL TOTALS
        quarter_row['Total_Amount'] = total_amount_all_types
        quarter_row['VOBU_EFTA_Amount'] = vobu_efta_amount_all_types
        
        agg_data.append(quarter_row)
    
    # Convert to DataFrame
    df_result = pd.DataFrame(agg_data)
    
    # Add total row
    if len(df_result) > 0:
        total_row = create_total_row(df_type, df_result, payment_type_name, reporting_year)
        df_result = pd.concat([df_result, total_row], ignore_index=True)
    
    return df_result

def create_total_row(df_type, df_result, payment_type_name, reporting_year):
    """
    Create total row for the aggregation table with VOBU/EFTA logic and transaction counts by call type
    """
    
    total_row = {
        'Quarter': 'Total',
        'Quarter_Short': 'Total',
        'Year': reporting_year,
        'Payment_Type': payment_type_name,
        'Reporting_Year': reporting_year
    }
    
    # Sum all amount columns (exclude Total row if it exists)
    df_data_only = df_result[df_result['Quarter'] != 'Total']
    
    # Sum individual call type amounts
    amount_cols = [col for col in df_result.columns if 'Amount' in col and col not in ['Total_Amount', 'VOBU_EFTA_Amount']]
    for col in amount_cols:
        total_row[col] = df_data_only[col].sum()
    
    # Calculate VOBU/EFTA total from original data (not summing quarterly totals to avoid double counting)
    df_vobu_efta = df_type[df_type['Fund Source'].isin(['VOBU', 'EFTA'])]
    total_row['VOBU_EFTA_Amount'] = df_vobu_efta['v_amount_to_sum'].sum()
    
    # Calculate transaction counts by call type from original data
    call_types = df_type['Call_Type_Display'].unique()
    for call_type in call_types:
        df_call_type = df_type[df_type['Call_Type_Display'] == call_type]
        total_row[f'No_of_Transactions_{call_type}'] = df_call_type['Pay Payment Key'].nunique()
    
    # Overall total amount
    total_row['Total_Amount'] = df_type['v_amount_to_sum'].sum()
    
    # Sum unique transactions across all quarters (deduplicated at total level)
    total_row['No_of_Transactions'] = df_type['Pay Payment Key'].nunique()
    
    return pd.DataFrame([total_row])

def format_table_for_great_tables(df_table, payment_type, programme, repeat_quarter=True):
    """
    Format table for great_tables library - creates clean pandas DataFrame
    Structure exactly like Excel: Quarter | Metric | ADG | COG | POC | STG | SYG | Total
    
    Args:
        repeat_quarter (bool): If True, repeat quarter value in each row. If False, show only once per group.
                              True is recommended for great_tables compatibility.
    """
    
    if len(df_table) == 0:
        return pd.DataFrame()
    
    # Separate data rows from total row
    df_data = df_table[df_table['Quarter'] != 'Total'].copy()
    df_total = df_table[df_table['Quarter'] == 'Total'].copy()
    
    if len(df_data) == 0:
        return pd.DataFrame()
    
    # Get unique quarters and call types from the data
    quarters = sorted(df_data['Quarter_Short'].unique())
    
    # Extract call type columns from the dataframe
    call_type_cols = [col for col in df_data.columns if col.startswith('Total_Amount_') and not col.endswith('Amount')]
    call_types = sorted([col.replace('Total_Amount_', '') for col in call_type_cols if col != 'Total_Amount'])
    
    print(f"  Formatting for great_tables - Call types: {call_types}, Quarters: {quarters}")
    print(f"  Quarter repeat mode: {repeat_quarter}")
    
    # Create the structure for great_tables - Quarter and Metric as separate columns
    table_data = []
    
    # === PROCESS EACH QUARTER ===
    for quarter in quarters:
        quarter_data = df_data[df_data['Quarter_Short'] == quarter]
        
        if len(quarter_data) == 0:
            continue
            
        # ROW 1: Total Amount for this quarter
        total_amount_row = {
            'Quarter': quarter, 
            'Metric': 'Total Amount'
        }
        
        for call_type in call_types:
            amount_col = f'Total_Amount_{call_type}'
            total_amount_row[call_type] = quarter_data[amount_col].iloc[0] if amount_col in quarter_data.columns else 0
        
        total_amount_row['Total'] = quarter_data['Total_Amount'].iloc[0]
        table_data.append(total_amount_row)
        
        # ROW 2: Out of Which VOBU/EFTA for this quarter
        vobu_efta_row = {
            'Quarter': quarter if repeat_quarter else '', 
            'Metric': 'Out of Which VOBU/EFTA'
        }
        
        for call_type in call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            vobu_efta_row[call_type] = quarter_data[vobu_efta_col].iloc[0] if vobu_efta_col in quarter_data.columns else 0
        
        vobu_efta_row['Total'] = quarter_data['VOBU_EFTA_Amount'].iloc[0]
        table_data.append(vobu_efta_row)
        
        # ROW 3: No of Transactions for this quarter
        transactions_row = {
            'Quarter': quarter if repeat_quarter else '', 
            'Metric': 'No of Transactions'
        }
        
        for call_type in call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            transactions_row[call_type] = quarter_data[transactions_col].iloc[0] if transactions_col in quarter_data.columns else 0
        
        transactions_row['Total'] = quarter_data['No_of_Transactions'].iloc[0]
        table_data.append(transactions_row)
    
    # === TOTAL ROWS (from df_total) ===
    if len(df_total) > 0:
        
        # TOTAL ROW 1: Total Amount
        total_amount_row = {
            'Quarter': 'Total', 
            'Metric': 'Total Amount'
        }
        
        for call_type in call_types:
            amount_col = f'Total_Amount_{call_type}'
            total_amount_row[call_type] = df_total[amount_col].iloc[0] if amount_col in df_total.columns else 0
        
        total_amount_row['Total'] = df_total['Total_Amount'].iloc[0]
        table_data.append(total_amount_row)
        
        # TOTAL ROW 2: Out of Which VOBU/EFTA
        total_vobu_efta_row = {
            'Quarter': 'Total' if repeat_quarter else '', 
            'Metric': 'Out of Which VOBU/EFTA'
        }
        
        for call_type in call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            total_vobu_efta_row[call_type] = df_total[vobu_efta_col].iloc[0] if vobu_efta_col in df_total.columns else 0
        
        total_vobu_efta_row['Total'] = df_total['VOBU_EFTA_Amount'].iloc[0]
        table_data.append(total_vobu_efta_row)
        
        # TOTAL ROW 3: No of Transactions
        total_transactions_row = {
            'Quarter': 'Total' if repeat_quarter else '', 
            'Metric': 'No of Transactions'
        }
        
        for call_type in call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            total_transactions_row[call_type] = df_total[transactions_col].iloc[0] if transactions_col in df_total.columns else 0
        
        total_transactions_row['Total'] = df_total['No_of_Transactions'].iloc[0]
        table_data.append(total_transactions_row)
    
    # Convert to DataFrame
    great_tables_df = pd.DataFrame(table_data)
    
    # Reorder columns: Quarter, Metric, then call types in alphabetical order, then Total
    column_order = ['Quarter', 'Metric'] + call_types + ['Total']
    great_tables_df = great_tables_df[column_order]
    
    return great_tables_df

# Main execution function with periodicity integration
def generate_all_quarterly_tables(df_paym, cutoff_date=None):
    """
    Main function to generate all quarterly payment tables with proper periodicity handling
    """
    
    print("Starting quarterly table generation with periodicity logic...")
    
    if cutoff_date is not None:
        print(f"Using provided cutoff date: {cutoff_date}")
    else:
        cutoff_date = pd.Timestamp.now()
        print(f"Using current date as cutoff: {cutoff_date}")
    
    # Generate tables with scope filtering
    results = create_quarterly_payment_tables(df_paym, cutoff_date)
    
    if results is None:
        return None
    
    # Format for display
    formatted_results = format_quarterly_tables_for_display(results)
    
    # Display summary
    print("\n=== GENERATION COMPLETE ===")
    print(f"Reporting for: {results['metadata']['reporting_year']}")
    print(f"Scope: {results['metadata']['scope_start']} to {results['metadata']['scope_end']}")
    
    if 'tables' in results:
        for programme, tables in results['tables'].items():
            print(f"\n{programme} Programme:")
            for payment_type, table in tables.items():
                data_rows = len(table[table['Quarter'] != 'Total']) if len(table) > 0 else 0
                print(f"  - {payment_type}: {data_rows} quarters")
    
    return formatted_results

def format_quarterly_tables_for_great_tables(results):
    """
    Format the results for great_tables library - clean pandas DataFrames
    """
    
    if 'tables' not in results:
        return results
        
    formatted_results = {
        'metadata': results['metadata'],
        'great_tables': {}
    }
    
    for programme, tables in results['tables'].items():
        formatted_results['great_tables'][programme] = {}
        
        for payment_type, df_table in tables.items():
            # Create great_tables format
            gt_table = format_table_for_great_tables(df_table, payment_type, programme)
            formatted_results['great_tables'][programme][payment_type] = gt_table
    
    return formatted_results

# Main execution function with great_tables output
def generate_all_quarterly_tables(df_paym, cutoff_date=None):
    """
    Main function to generate all quarterly payment tables for great_tables
    """
    
    print("Starting quarterly table generation for great_tables...")
    
    if cutoff_date is not None:
        print(f"Using provided cutoff date: {cutoff_date}")
    else:
        cutoff_date = pd.Timestamp.now()
        print(f"Using current date as cutoff: {cutoff_date}")
    
    # Generate tables with scope filtering
    results = create_quarterly_payment_tables(df_paym, cutoff_date)
    
    if results is None:
        return None
    
    # Format for great_tables
    formatted_results = format_quarterly_tables_for_great_tables(results)
    
    # Display summary
    print("\n=== GENERATION COMPLETE ===")
    print(f"Reporting for: {results['metadata']['reporting_year']}")
    print(f"Scope: {results['metadata']['scope_start']} to {results['metadata']['scope_end']}")
    print(f"VOBU/EFTA aggregation: Only EFTA and VOBU fund sources included")
    
    if 'tables' in results:
        for programme, tables in results['tables'].items():
            print(f"\n{programme} Programme:")
            for payment_type, table in tables.items():
                data_rows = len(table[table['Quarter'] != 'Total']) if len(table) > 0 else 0
                print(f"  - {payment_type}: {data_rows} quarters")
    
    return formatted_results

# Updated utility functions for great_tables
def get_great_table(formatted_results, programme, payment_type):
    """
    Get a specific table formatted for great_tables
    """
    try:
        return formatted_results['great_tables'][programme][payment_type]
    except KeyError:
        print(f"Table not found: {programme} - {payment_type}")
        available_programmes = list(formatted_results.get('great_tables', {}).keys())
        print(f"Available programmes: {available_programmes}")
        if programme in formatted_results.get('great_tables', {}):
            available_payment_types = list(formatted_results['great_tables'][programme].keys())
            print(f"Available payment types for {programme}: {available_payment_types}")
        return pd.DataFrame()

def get_summary_table(formatted_results, programme):
    """
    Get the summary table that includes all payment types (including experts)
    """
    return get_great_table(formatted_results, programme, 'All_Payments')

def create_comprehensive_summary_table(results, programme):
    """
    Create a comprehensive summary table showing all payment types in one view
    """
    
    if 'tables' not in results or programme not in results['tables']:
        print(f"No data found for programme: {programme}")
        return pd.DataFrame()
    
    programme_tables = results['tables'][programme]
    
    # Initialize summary data
    summary_data = []
    
    # Get all call types from any table
    all_call_types = set()
    for payment_type, table in programme_tables.items():
        if payment_type != 'All_Payments' and len(table) > 0:
            total_row = table[table['Quarter'] == 'Total']
            if len(total_row) > 0:
                call_type_cols = [col for col in total_row.columns if col.startswith('Total_Amount_')]
                call_types = [col.replace('Total_Amount_', '') for col in call_type_cols]
                all_call_types.update(call_types)
    
    all_call_types = sorted(list(all_call_types))
    
    # Create rows for each payment type
    for payment_type, table in programme_tables.items():
        if payment_type == 'All_Payments':
            continue  # Skip the existing all payments, we'll create our own
            
        if len(table) == 0:
            continue
            
        total_row = table[table['Quarter'] == 'Total']
        if len(total_row) == 0:
            continue
            
        # === TOTAL AMOUNT ROW ===
        amount_row = {'Payment_Type': payment_type, 'Metric': 'Total Amount'}
        
        for call_type in all_call_types:
            amount_col = f'Total_Amount_{call_type}'
            amount_row[call_type] = total_row[amount_col].iloc[0] if amount_col in total_row.columns else 0
        
        amount_row['Total'] = total_row['Total_Amount'].iloc[0]
        summary_data.append(amount_row)
        
        # === VOBU/EFTA ROW ===
        vobu_efta_row = {'Payment_Type': payment_type, 'Metric': 'Out of Which VOBU/EFTA'}
        
        for call_type in all_call_types:
            vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
            vobu_efta_row[call_type] = total_row[vobu_efta_col].iloc[0] if vobu_efta_col in total_row.columns else 0
        
        vobu_efta_row['Total'] = total_row['VOBU_EFTA_Amount'].iloc[0]
        summary_data.append(vobu_efta_row)
        
        # === TRANSACTIONS ROW ===
        transactions_row = {'Payment_Type': payment_type, 'Metric': 'No of Transactions'}
        
        for call_type in all_call_types:
            transactions_col = f'No_of_Transactions_{call_type}'
            transactions_row[call_type] = total_row[transactions_col].iloc[0] if transactions_col in total_row.columns else 0
        
        transactions_row['Total'] = total_row['No_of_Transactions'].iloc[0]
        summary_data.append(transactions_row)
    
    # === CREATE OVERALL TOTALS ===
    if summary_data:
        # Get the All_Payments table data
        all_payments_table = programme_tables.get('All_Payments', pd.DataFrame())
        
        if len(all_payments_table) > 0:
            total_row = all_payments_table[all_payments_table['Quarter'] == 'Total']
            
            if len(total_row) > 0:
                # TOTAL AMOUNTS ACROSS ALL PAYMENT TYPES
                total_amount_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'Total Amount'}
                for call_type in all_call_types:
                    amount_col = f'Total_Amount_{call_type}'
                    total_amount_row[call_type] = total_row[amount_col].iloc[0] if amount_col in total_row.columns else 0
                total_amount_row['Total'] = total_row['Total_Amount'].iloc[0]
                summary_data.append(total_amount_row)
                
                # TOTAL VOBU/EFTA ACROSS ALL PAYMENT TYPES
                total_vobu_efta_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'Out of Which VOBU/EFTA'}
                for call_type in all_call_types:
                    vobu_efta_col = f'VOBU_EFTA_Amount_{call_type}'
                    total_vobu_efta_row[call_type] = total_row[vobu_efta_col].iloc[0] if vobu_efta_col in total_row.columns else 0
                total_vobu_efta_row['Total'] = total_row['VOBU_EFTA_Amount'].iloc[0]
                summary_data.append(total_vobu_efta_row)
                
                # TOTAL TRANSACTIONS ACROSS ALL PAYMENT TYPES (deduplicated)
                total_transactions_row = {'Payment_Type': 'TOTAL ALL TYPES', 'Metric': 'No of Transactions'}
                for call_type in all_call_types:
                    transactions_col = f'No_of_Transactions_{call_type}'
                    total_transactions_row[call_type] = total_row[transactions_col].iloc[0] if transactions_col in total_row.columns else 0
                total_transactions_row['Total'] = total_row['No_of_Transactions'].iloc[0]
                summary_data.append(total_transactions_row)
    
    # Convert to DataFrame
    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        
        # Reorder columns
        column_order = ['Payment_Type', 'Metric'] + all_call_types + ['Total']
        summary_df = summary_df[column_order]
        
        return summary_df
    else:
        return pd.DataFrame()

def create_payment_type_comparison_table(results, programme):
    """
    Create a table showing just the totals for each payment type for easy comparison
    """
    
    if 'tables' not in results or programme not in results['tables']:
        return pd.DataFrame()
    
    programme_tables = results['tables'][programme]
    comparison_data = []
    
    for payment_type, table in programme_tables.items():
        if payment_type == 'All_Payments':
            continue
            
        if len(table) == 0:
            continue
            
        total_row = table[table['Quarter'] == 'Total']
        if len(total_row) == 0:
            continue
        
        comparison_row = {
            'Payment_Type': payment_type,
            'Total_Amount': total_row['Total_Amount'].iloc[0],
            'VOBU_EFTA_Amount': total_row['VOBU_EFTA_Amount'].iloc[0],
            'No_of_Transactions': total_row['No_of_Transactions'].iloc[0]
        }
        
        comparison_data.append(comparison_row)
    
    # Add overall total
    if comparison_data:
        all_payments_table = programme_tables.get('All_Payments', pd.DataFrame())
        if len(all_payments_table) > 0:
            total_row = all_payments_table[all_payments_table['Quarter'] == 'Total']
            if len(total_row) > 0:
                overall_row = {
                    'Payment_Type': 'TOTAL ALL TYPES',
                    'Total_Amount': total_row['Total_Amount'].iloc[0],
                    'VOBU_EFTA_Amount': total_row['VOBU_EFTA_Amount'].iloc[0],
                    'No_of_Transactions': total_row['No_of_Transactions'].iloc[0]
                }
                comparison_data.append(overall_row)
    
    return pd.DataFrame(comparison_data)

def list_available_tables(formatted_results):
    """
    List all available tables including summary options
    """
    print("=== AVAILABLE TABLES FOR GREAT_TABLES ===")
    
    if 'tables' not in formatted_results:
        print("No tables found")
        return
    
    for programme, tables in formatted_results['tables'].items():
        print(f"\n{programme} Programme:")
        for payment_type, df_table in tables.items():
            rows, cols = df_table.shape
            if payment_type == 'All_Payments':
                print(f"  - {payment_type}: {rows} rows x {cols} columns ⭐ SUMMARY TABLE")
            else:
                print(f"  - {payment_type}: {rows} rows x {cols} columns")
        
        print(f"\n  📊 Access functions available:")
        print(f"    # Individual payment types:")
        print(f"    get_great_table(results, '{programme}', 'Pre-financing', repeat_quarter=True)  # Recommended")
        print(f"    get_great_table_repeated(results, '{programme}', 'Pre-financing')  # Same as above")
        print(f"    get_great_table_grouped(results, '{programme}', 'Pre-financing')   # Excel visual style")
        print(f"    ")
        print(f"    # Summary tables:")
        print(f"    get_summary_table(results, '{programme}', repeat_quarter=True)  # All payment types")
        print(f"    create_comprehensive_summary_table(results, '{programme}')       # Alternative")
        print(f"    create_payment_type_comparison_table(results, '{programme}')     # Quick comparison")

def get_all_programme_tables(formatted_results, programme):
    """
    Get all tables for a specific programme as a dictionary
    """
    try:
        return formatted_results['tables'][programme]
    except KeyError:
        print(f"Programme not found: {programme}")
        available = list(formatted_results.get('tables', {}).keys())
        print(f"Available programmes: {available}")
        return {}

def combine_payment_types_table(formatted_results, programme):
    """
    Combine all payment types for a programme into one large table
    """
    programme_tables = get_all_programme_tables(formatted_results, programme)
    
    if not programme_tables:
        return pd.DataFrame()
    
    combined_tables = []
    
    for payment_type, df_table in programme_tables.items():
        if len(df_table) > 0:
            # Add a separator row if not the first table
            if len(combined_tables) > 0:
                separator_row = pd.DataFrame([{
                    'Metric': f'--- {payment_type} ---',
                    **{col: '' for col in df_table.columns if col != 'Metric'}
                }])
                combined_tables.append(separator_row)
            
            combined_tables.append(df_table)
    
    if combined_tables:
        return pd.concat(combined_tables, ignore_index=True)
    else:
        return pd.DataFrame()

# Example usage functions
def show_table_summary_for_great_tables(formatted_results):
    """Display summary of all generated tables for great_tables"""
    print("=== QUARTERLY TABLES SUMMARY FOR GREAT_TABLES ===")
    print(f"Reporting Period: {formatted_results['metadata']['reporting_year']}")
    print(f"Data Range: {formatted_results['metadata']['actual_date_range']['start'].strftime('%Y-%m-%d')} to {formatted_results['metadata']['actual_date_range']['end'].strftime('%Y-%m-%d')}")
    print(f"VOBU/EFTA Logic: Aggregates only EFTA and VOBU fund sources")
    
    if 'great_tables' in formatted_results:
        for programme, tables in formatted_results['great_tables'].items():
            print(f"\n{programme}:")
            for payment_type, table in tables.items():
                if len(table) > 0:
                    # Get total amounts from the 'Total Amount' row
                    total_amount_row = table[table['Metric'] == 'Total Amount']
                    vobu_efta_row = table[table['Metric'] == 'Out of Which VOBU/EFTA']
                    transactions_row = table[table['Metric'] == 'No of Transactions']
                    
                    if len(total_amount_row) > 0:
                        total_amount = total_amount_row['Total'].iloc[0]
                        vobu_efta_amount = vobu_efta_row['Total'].iloc[0] if len(vobu_efta_row) > 0 else 0
                        total_transactions = transactions_row['Total'].iloc[0] if len(transactions_row) > 0 else 0
                        print(f"  {payment_type}: €{total_amount:,.0f} total (€{vobu_efta_amount:,.0f} VOBU/EFTA) - {total_transactions} transactions")

# Updated access examples
# Backwards compatibility functions
def get_excel_format_table(formatted_results, programme, payment_type):
    """
    Backwards compatibility function - redirects to get_great_table
    """
    print("Note: get_excel_format_table is deprecated, use get_great_table for great_tables formatting")
    return get_great_table(formatted_results, programme, payment_type)

def show_table_summary(formatted_results):
    """
    Backwards compatibility function - redirects to show_table_summary_for_great_tables
    """
    return show_table_summary_for_great_tables(formatted_results)

# Updated access examples with summary tables
def print_usage_instructions():
    """
    Print instructions for using the generated tables including summary tables
    """
    print("""
=== HOW TO USE THE QUARTERLY TABLES (Two Format Options) ===

1. GENERATE TABLES:
   quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff)

2. ACCESS INDIVIDUAL PAYMENT TYPE TABLES:
   
   # OPTION A: Repeated quarters (RECOMMENDED for great_tables)
   heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=True)
   heu_interim = get_great_table(quarterly_tables, 'HEU', 'Interim Payments', repeat_quarter=True)
   
   # OPTION B: Grouped quarters (Excel visual style)
   heu_prefinancing_grouped = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=False)
   
   # Shortcut functions:
   heu_prefinancing = get_great_table_repeated(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option A
   heu_prefinancing_grouped = get_great_table_grouped(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option B

3. ACCESS SUMMARY TABLES (ALL PAYMENT TYPES INCLUDING EXPERTS):
   
   # Summary table with repeated quarters (recommended for great_tables)
   heu_summary = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
   
   # Summary table with grouped quarters (Excel visual style)
   heu_summary_grouped = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=False)
   
   # Alternative access
   heu_summary_alt = create_comprehensive_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
   
   # Quick comparison table (different format - just totals by payment type)
   heu_comparison = create_payment_type_comparison_table(quarterly_tables, 'HEU')

4. LIST ALL AVAILABLE TABLES:
   list_available_tables(quarterly_tables)

5. USE WITH GREAT_TABLES:
   from great_tables import GT
   
   # Individual payment type (using repeated format)
   gt_table = (
       GT(heu_prefinancing)
       .tab_header(title="HEU Pre-financing", subtitle="Q1 2025")
       .fmt_currency(columns=['ADG', 'COG', 'POC', 'STG', 'SYG', 'Total'], currency='EUR')
       .tab_row_group(label="2024Q1", rows=["2024Q1"])  # Can group by Quarter if using repeated format
   )

FORMAT OPTIONS:

OPTION A - Repeated Quarters (RECOMMENDED for great_tables):
Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
2024Q1  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
2024Q1  | No of Transactions      | ... | ... | ... | ... | ... | ...
2024Q2  | Total Amount            | ... | ... | ... | ... | ... | ...
2024Q2  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
2024Q2  | No of Transactions      | ... | ... | ... | ... | ... | ...

OPTION B - Grouped Quarters (Excel visual style):
Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
        | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
        | No of Transactions      | ... | ... | ... | ... | ... | ...
2024Q2  | Total Amount            | ... | ... | ... | ... | ... | ...
        | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
        | No of Transactions      | ... | ... | ... | ... | ... | ...

RECOMMENDATION:
✅ Use repeat_quarter=True (Option A) for great_tables - better for:
  - Data processing and filtering
  - Row grouping operations
  - Exporting to other formats
  - Table manipulation

Use repeat_quarter=False (Option B) for:
  - Pure visual presentation matching Excel
  - When you want minimal visual clutter
""")

def show_summary_examples():
    """
    Show examples of the different summary table options with correct format
    """
    print("""
=== SUMMARY TABLE EXAMPLES (Both Format Options) ===

# 1a. Summary table with repeated quarters (RECOMMENDED for great_tables)
heu_summary = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=True)
# Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
# 2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
# 2024Q1  | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
# 2024Q1  | No of Transactions      | ... | ... | ... | ... | ... | ...
# Total   | Total Amount            | ... | ... | ... | ... | ... | ...
# Total   | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
# Total   | No of Transactions      | ... | ... | ... | ... | ... | ...

# 1b. Summary table with grouped quarters (Excel visual style)
heu_summary_grouped = get_summary_table(quarterly_tables, 'HEU', repeat_quarter=False)
# Quarter | Metric                  | ADG | COG | POC | STG | SYG | Total
# 2024Q1  | Total Amount            | ... | ... | ... | ... | ... | ...
#         | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
#         | No of Transactions      | ... | ... | ... | ... | ... | ...
# Total   | Total Amount            | ... | ... | ... | ... | ... | ...
#         | Out of Which VOBU/EFTA  | ... | ... | ... | ... | ... | ...
#         | No of Transactions      | ... | ... | ... | ... | ... | ...

# 2. Quick comparison (different format - by payment type)
heu_comparison = create_payment_type_comparison_table(quarterly_tables, 'HEU')
# Payment_Type | Total_Amount | VOBU_EFTA_Amount | No_of_Transactions
# Pre-financing | 352,568,326 | 336,797,403 | 309
# Interim Payments | 185,710,686 | 175,320,845 | 115
# TOTAL ALL TYPES | 1,628,225,910 | 1,398,938,052 | 1,573

BOTH formats contain the same data, just different visual presentation!
""")

# If run directly, show instructions
if __name__ == "__main__":
    print_usage_instructions()
    print("\n" + "="*60 + "\n")
    show_summary_examples()


=== HOW TO USE THE QUARTERLY TABLES (Two Format Options) ===

1. GENERATE TABLES:
   quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff)

2. ACCESS INDIVIDUAL PAYMENT TYPE TABLES:

   # OPTION A: Repeated quarters (RECOMMENDED for great_tables)
   heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=True)
   heu_interim = get_great_table(quarterly_tables, 'HEU', 'Interim Payments', repeat_quarter=True)

   # OPTION B: Grouped quarters (Excel visual style)
   heu_prefinancing_grouped = get_great_table(quarterly_tables, 'HEU', 'Pre-financing', repeat_quarter=False)

   # Shortcut functions:
   heu_prefinancing = get_great_table_repeated(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option A
   heu_prefinancing_grouped = get_great_table_grouped(quarterly_tables, 'HEU', 'Pre-financing')  # Same as Option B

3. ACCESS SUMMARY TABLES (ALL PAYMENT TYPES INCLUDING EXPERTS):

   # Summary table with repeated quarters (recommended for 

In [8]:
# Example usage:
quarterly_tables = generate_all_quarterly_tables(df_paym, cutoff )


Starting quarterly table generation for great_tables...
Using provided cutoff date: 2025-04-15 00:00:00
=== QUARTERLY PAYMENT TABLES GENERATION ===
Cutoff date: 2025-04-15 00:00:00
Reporting year: 2025
Expected scope: 2025-01-01 00:00:00 to 2025-03-31 00:00:00
Note: Assuming df_paym is already filtered for this scope
Found call type column: call_type
✓ All required columns present
Working dataset: 2506 rows
Actual date range: 2025-01-16 00:00:00 to 2025-03-31 00:00:00
Quarters found: ['2025Q1']
Call types found: ['ADG', 'COG', 'EXPERTS', 'POC', 'STG', 'SYG']
Programmes found: ['HEU' 'H2020']

=== Processing HEU ===
  Creating table for: Pre-financing
  Creating table for: Interim Payments
  Creating table for: Experts and Support
  Creating table for: Final Payments
  Creating overall summary table

=== Processing H2020 ===
  Creating table for: Interim Payments
  Creating table for: Final Payments
  Creating overall summary table
  Formatting for great_tables - Call types: ['ADG', 'CO

In [10]:
heu_prefinancing = get_great_table(quarterly_tables, 'HEU', 'Pre-financing')
heu_prefinancing

,Quarter,Metric,ADG,COG,POC,STG,SYG,Total
0,1Q2025,Total Amount,60065912.4,83341828.25,5062500.0,1.247404e+08,79357691.7,3.525683e+08
1,1Q2025,Out of Which VOBU/EFTA,60065912.4,79520778.55,4965000.0,1.128881e+08,79357691.7,3.367974e+08
2,1Q2025,No of Transactions,37.0,65.00,45.0,1.390000e+02,23.0,3.090000e+02
3,Total,Total Amount,60065912.4,83341828.25,5062500.0,1.247404e+08,79357691.7,3.525683e+08
4,Total,Out of Which VOBU/EFTA,60065912.4,79520778.55,4965000.0,1.128881e+08,79357691.7,3.367974e+08
5,Total,No of Transactions,37.0,65.00,45.0,1.390000e+02,23.0,3.090000e+02


In [11]:
heu_ip = get_great_table(quarterly_tables, 'HEU', 'Interim Payments')
heu_ip

,Quarter,Metric,ADG,COG,STG,SYG,Total
0,1Q2025,Total Amount,5671359.26,8458523.99,29907014.43,21033322.99,65070220.67
1,1Q2025,Out of Which VOBU/EFTA,5671359.26,7958749.99,25046321.66,19743560.76,58419991.67
2,1Q2025,No of Transactions,10.00,17.00,93.00,14.00,134.00
3,Total,Total Amount,5671359.26,8458523.99,29907014.43,21033322.99,65070220.67
4,Total,Out of Which VOBU/EFTA,5671359.26,7958749.99,25046321.66,19743560.76,58419991.67
5,Total,No of Transactions,10.00,17.00,93.00,14.00,134.00


In [12]:
heu_fp = get_great_table(quarterly_tables, 'HEU', 'Final Payments')
heu_fp

,Quarter,Metric,POC,Total
0,1Q2025,Total Amount,1632000.00,1632000.00
1,1Q2025,Out of Which VOBU/EFTA,1541863.75,1541863.75
2,1Q2025,No of Transactions,58.00,58.00
3,Total,Total Amount,1632000.00,1632000.00
4,Total,Out of Which VOBU/EFTA,1541863.75,1541863.75
5,Total,No of Transactions,58.00,58.00


In [15]:
heu_expt = get_great_table(quarterly_tables, 'HEU', 'Experts and Support')
heu_expt

,Quarter,Metric,EXPERTS,Total
0,1Q2025,Total Amount,3564146.73,3564146.73
1,1Q2025,Out of Which VOBU/EFTA,3519456.41,3519456.41
2,1Q2025,No of Transactions,1378.00,1378.00
3,Total,Total Amount,3564146.73,3564146.73
4,Total,Out of Which VOBU/EFTA,3519456.41,3519456.41
5,Total,No of Transactions,1378.00,1378.00


In [17]:
heu_all = get_great_table(quarterly_tables, 'HEU', 'All_Payments')
heu_all

,Quarter,Metric,ADG,COG,EXPERTS,POC,STG,SYG,Total
0,1Q2025,Total Amount,65737271.66,91800352.24,3564146.73,6694500.00,1.546474e+08,1.003910e+08,4.228347e+08
1,1Q2025,Out of Which VOBU/EFTA,65737271.66,87479528.54,3519456.41,6506863.75,1.379344e+08,9.910125e+07,4.002787e+08
2,1Q2025,No of Transactions,47.00,82.00,1378.00,103.00,2.320000e+02,3.700000e+01,1.879000e+03
3,Total,Total Amount,65737271.66,91800352.24,3564146.73,6694500.00,1.546474e+08,1.003910e+08,4.228347e+08
4,Total,Out of Which VOBU/EFTA,65737271.66,87479528.54,3519456.41,6506863.75,1.379344e+08,9.910125e+07,4.002787e+08
5,Total,No of Transactions,47.00,82.00,1378.00,103.00,2.320000e+02,3.700000e+01,1.879000e+03


In [ ]:
db_path = "database/reporting.db"
DB_PATH = Path("database/reporting.db")
report = 'Quarterly_Report'
report_params = load_report_params(report_name=report, db_path=DB_PATH )
TTP_gross = report_params.get("TTP_GROSS_HISTORY")
TTP_gross_H2020 = TTP_gross.get('H2020')


{'IP': 74.2, 'FP': 78.2, 'Experts': 148, 'H2020': 75.6}

In [ ]:
import pandas as pd
from pathlib import Path

# =============================================================================
# CLEAN TTP CALCULATION FUNCTIONS
# =============================================================================

def calculate_current_ttp_metrics(df_paym, cutoff):
    """
    Calculate current TTP metrics from df_paym data
    """
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Workflow Last AOS Action Date'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
    
    results = {}
    
    # Calculate by Programme and Payment Type
    for programme in ['H2020', 'HEU']:
        prog_data = df_unique[df_unique['Programme'] == programme]
        if len(prog_data) == 0:
            continue
            
        results[programme] = {}
        
        # Overall programme metrics
        prog_valid = prog_data[prog_data['v_payment_in_time'].notna()]
        results[programme]['overall'] = {
            'avg_ttp_net': prog_data['v_TTP_NET'].mean(),
            'avg_ttp_gross': prog_data['v_TTP_GROSS'].mean(),
            'on_time_pct': prog_data['v_payment_in_time'].sum() / len(prog_valid) if len(prog_valid) > 0 else 0
        }
        
        # By payment type - using correct short form values from v_payment_type
        payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
        
        for payment_type in payment_types:
            pt_data = prog_data[prog_data['v_payment_type'] == payment_type]
            if len(pt_data) > 0:
                pt_valid = pt_data[pt_data['v_payment_in_time'].notna()]
                results[programme][payment_type] = {
                    'avg_ttp_net': pt_data['v_TTP_NET'].mean(),
                    'avg_ttp_gross': pt_data['v_TTP_GROSS'].mean(),
                    'on_time_pct': pt_data['v_payment_in_time'].sum() / len(pt_valid) if len(pt_valid) > 0 else 0
                }
    
    # Overall total
    total_valid = df_unique[df_unique['v_payment_in_time'].notna()]
    results['TOTAL'] = {
        'avg_ttp_net': df_unique['v_TTP_NET'].mean(),
        'avg_ttp_gross': df_unique['v_TTP_GROSS'].mean(),
        'on_time_pct': df_unique['v_payment_in_time'].sum() / len(total_valid) if len(total_valid) > 0 else 0
    }
    
    return results

def load_historical_ttp_data(report_name='Quarterly_Report', db_path="database/reporting.db"):
    """
    Load historical TTP data from database
    """
    DB_PATH = Path(db_path)
    report_params = load_report_params(report_name=report_name, db_path=DB_PATH)
    
    return {
        "TTP_NET_HISTORY": report_params.get("TTP_NET_HISTORY"),
        "TTP_GROSS_HISTORY": report_params.get("TTP_GROSS_HISTORY"),
        "PAYMENTS_ON_TIME_HISTORY": report_params.get("PAYMENTS_ON_TIME_HISTORY")
    }

def create_ttp_comparison_table(df_paym, cutoff, historical_data):
    """
    Create TTP comparison table matching the image structure
    """
    # Calculate current metrics
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    
    # Determine labels based on cutoff
    cutoff_date = pd.to_datetime(cutoff)
    current_year = cutoff_date.year
    current_label = f"{current_year}-YTD"
    historical_label = f"Dec {current_year - 1}"
    
    # Build comparison data
    comparison_data = []
    
    # H2020 section
    h2020_current = current_metrics.get('H2020', {})
    
    # H2020 - Interim Payments (IP)
    current_ip = h2020_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('IP', 0)*100:.2f}%"
    })
    
    # H2020 - Final Payments (FP)
    current_fp = h2020_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('FP', 0)*100:.2f}%"
    })
    
    # H2020 - Experts Payments (EXPERTS)
    current_exp = h2020_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('Experts', 0)*100:.2f}%"
    })
    
    # H2020 - Overall
    current_h2020 = h2020_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'H2020',
        f'Average Net Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_h2020['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["H2020"].get("H2020", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_h2020['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['H2020'].get('H2020', 0)*100:.2f}%"
    })
    
    # HEU section
    heu_current = current_metrics.get('HEU', {})
    
    # HEU - Prefinancing Payments (PF)
    current_pf = heu_current.get('PF', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Prefinancing Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_pf['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("PF", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_pf['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('PF', 0)*100:.2f}%"
    })
    
    # HEU - Interim Payments (IP)
    current_ip_heu = heu_current.get('IP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Interim Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_ip_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("IP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_ip_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('IP', 0)*100:.2f}%"
    })
    
    # HEU - Final Payments (FP)
    current_fp_heu = heu_current.get('FP', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Final Payments',
        f'Average Net Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_fp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("FP", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_fp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('FP', 0)*100:.2f}%"
    })
    
    # HEU - Experts Payment (EXPERTS)
    current_exp_heu = heu_current.get('EXPERTS', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'Experts Payment',
        f'Average Net Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_exp_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("Experts", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_exp_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('Experts', 0)*100:.2f}%"
    })
    
    # HEU - Overall
    current_heu = heu_current.get('overall', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'HEU',
        f'Average Net Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_heu['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["HEU"].get("HEU", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_heu['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['HEU'].get('HEU', 0)*100:.2f}%"
    })
    
    # TOTAL row
    current_total = current_metrics.get('TOTAL', {'avg_ttp_net': 0, 'avg_ttp_gross': 0, 'on_time_pct': 0})
    comparison_data.append({
        'Type of Payments': 'TOTAL',
        f'Average Net Time to Pay (in days) {current_label}': round(current_total['avg_ttp_net'], 1),
        f'Average Net Time to Pay (in days) {historical_label}': historical_data["TTP_NET_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Average Gross Time to Pay (in days) {current_label}': round(current_total['avg_ttp_gross'], 1),
        f'Average Gross Time to Pay (in days) {historical_label}': historical_data["TTP_GROSS_HISTORY"]["ALL"].get("TOTAL", "n.a"),
        f'Target Paid on Time - Contractually {current_label}': f"{current_total['on_time_pct']*100:.2f}%",
        f'Target Paid on Time - Contractually {historical_label}': f"{historical_data['PAYMENTS_ON_TIME_HISTORY']['ALL'].get('TOTAL', 0)*100:.2f}%"
    })
    
    return pd.DataFrame(comparison_data)

# =============================================================================
# MAIN FUNCTION - USE THIS IN JUPYTER
# =============================================================================

def generate_ttp_comparison_table(df_paym, cutoff):
    """
    Main function to generate TTP comparison table
    
    Usage in Jupyter:
    ttp_table = generate_ttp_comparison_table(df_paym, cutoff)
    """
    # Load historical data
    historical_data = load_historical_ttp_data()
    
    # Create comparison table
    ttp_table = create_ttp_comparison_table(df_paym, cutoff, historical_data)
    
    return ttp_table



In [49]:
# =============================================================================
# USAGE IN JUPYTER NOTEBOOK
# =============================================================================


# Use this in your Jupyter notebook:
ttp_comparison_df = generate_ttp_comparison_table(df_paym, cutoff)
ttp_comparison_df 



,Type of Payments,Average Net Time to Pay (in days) 2025-YTD,Average Net Time to Pay (in days) Dec 2024,Average Gross Time to Pay (in days) 2025-YTD,Average Gross Time to Pay (in days) Dec 2024,Target Paid on Time - Contractually 2025-YTD,Target Paid on Time - Contractually Dec 2024
0,Interim Payments,36.3,22.1,53.4,74.2,99.67%,100.00%
1,Final Payments,63.6,48.0,117.7,78.2,91.10%,100.00%
2,Experts Payments,0.0,41.0,0.0,148.0,0.00%,0.00%
3,H2020,46.9,30.8,78.3,75.6,96.35%,99.96%
4,Prefinancing Payments,15.4,5.4,15.4,5.4,99.35%,99.33%
5,Interim Payments,29.4,15.0,39.3,17.7,100.00%,100.00%
6,Final Payments,51.7,42.7,57.2,47.6,98.28%,100.00%
7,Experts Payment,13.6,9.8,13.6,10.0,98.33%,99.43%
8,HEU,16.2,9.9,17.1,10.3,98.62%,99.43%
9,TOTAL,22.6,15.1,29.8,26.7,98.15%,99.60%


In [62]:
def calculate_current_ttp_metrics(df_paym, cutoff):
    """
    Calculate current TTP metrics from df_paym data
    """
    # Filter data up to cutoff and deduplicate by Pay Payment Key
    quarter_dates = get_scope_start_end(cutoff=cutoff)
    last_valid_date = quarter_dates[1]

    df_filtered = df_paym[
        df_paym['Pay Workflow Last AOS Action Date'] <= last_valid_date
    ].copy()
    df_unique = df_filtered.drop_duplicates(subset=['Pay Payment Key']).copy()
    
    # Convert to numeric
    df_unique['v_TTP_NET'] = pd.to_numeric(df_unique['v_TTP_NET'], errors='coerce')
    df_unique['v_TTP_GROSS'] = pd.to_numeric(df_unique['v_TTP_GROSS'], errors='coerce')
    df_unique['v_payment_in_time'] = pd.to_numeric(df_unique['v_payment_in_time'], errors='coerce')
    
    # Filter out negative v_TTP_NET values
    df_unique = df_unique[df_unique['v_TTP_NET'] >= 0]
    
    results = {}
    
    # Calculate by Programme and Payment Type
    for programme in ['H2020', 'HEU']:
        prog_data = df_unique[df_unique['Programme'] == programme]
        if len(prog_data) == 0:
            continue
            
        results[programme] = {}
        
        # Overall programme metrics
        prog_valid = prog_data[prog_data['v_payment_in_time'].notna()]
        results[programme]['overall'] = {
            'avg_ttp_net': prog_data['v_TTP_NET'].mean(),
            'avg_ttp_gross': prog_data['v_TTP_GROSS'].mean(),
            'on_time_pct': prog_data['v_payment_in_time'].sum() / len(prog_valid) if len(prog_valid) > 0 else 0
        }
        
        # By payment type - using correct short form values from v_payment_type
        payment_types = ['IP', 'FP', 'EXPERTS', 'PF']  # Short form values
        
        for payment_type in payment_types:
            pt_data = prog_data[prog_data['v_payment_type'] == payment_type]
            if len(pt_data) > 0:
                pt_valid = pt_data[pt_data['v_payment_in_time'].notna()]
                results[programme][payment_type] = {
                    'avg_ttp_net': pt_data['v_TTP_NET'].mean(),
                    'avg_ttp_gross': pt_data['v_TTP_GROSS'].mean(),
                    'on_time_pct': pt_data['v_payment_in_time'].sum() / len(pt_valid) if len(pt_valid) > 0 else 0
                }
    
    # Overall total
    total_valid = df_unique[df_unique['v_payment_in_time'].notna()]
    results['TOTAL'] = {
        'avg_ttp_net': df_unique['v_TTP_NET'].mean(),
        'avg_ttp_gross': df_unique['v_TTP_GROSS'].mean(),
        'on_time_pct': df_unique['v_payment_in_time'].sum() / len(total_valid) if len(total_valid) > 0 else 0
    }
    
    return results

def create_quarterly_ttp_table(df_paym, cutoff, programme, payment_type):
    """
    Create a quarterly TTP table for a specific programme and payment type
    """
    # Filter data
    df_filtered = df_paym[
        (df_paym['Pay Workflow Last AOS Action Date'] <= pd.to_datetime(cutoff)) &
        (df_paym['Programme'] == programme) &
        (df_paym['v_payment_type'] == payment_type)
    ].copy()
    
    # Convert to numeric and filter negative TTP_NET (handled in calculate_current_ttp_metrics)
    df_filtered['v_TTP_NET'] = pd.to_numeric(df_filtered['v_TTP_NET'], errors='coerce')
    df_filtered = df_filtered[df_filtered['v_TTP_NET'] >= 0]
    
    # Extract quarter from date
    df_filtered['Quarter'] = pd.to_datetime(df_filtered['Pay Workflow Last AOS Action Date']).dt.to_period('Q').astype(str)
    
    # Assume CallType exists or use a placeholder
    if 'CallType' not in df_filtered.columns:
        df_filtered['CallType'] = 'Default'  # Adjust if CallType has specific values
    
    # Aggregate by Quarter and CallType
    quarterly_data = df_filtered.groupby(['Quarter', 'CallType']).agg({
        'v_TTP_NET': 'mean'
    }).round(1).unstack(fill_value=0)
    
    # Rename columns to match call types (e.g., ADG, COG, STG from image)
    quarterly_data.columns = [f'{col[1]}' for col in quarterly_data.columns]
    
    # Add total row
    total_row = quarterly_data.mean().rename('TOTAL')
    quarterly_table = pd.concat([quarterly_data, total_row.to_frame().T])
    
    # Add programme and payment type to the index for clarity
    quarterly_table.index = [f"{programme} - {payment_type} - {idx}" for idx in quarterly_table.index]
    
    return quarterly_table

def chart_machine_ttp(df, payment_type, prog, avg, rolling_avg):
    """
    Generate TTP chart with monthly average and total average line
    """
    if payment_type == 'Prefinancing':
        d = {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'time_limit': [30] * 12}
        df_time_limit = pd.DataFrame(data=d)
    elif payment_type == 'Experts':
        d = {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'time_limit': [30] * 12}
        df_time_limit = pd.DataFrame(data=d)
    else:
        d = {'Month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'time_limit': [90] * 12}
        df_time_limit = pd.DataFrame(data=d)

    ttp_avg = avg
    total_avg = df['TTP_NET'].mean()  # Total average for the red line

    def my_theme():
        return {
            'config': {
                'view': {'continuousHeight': 300, 'continuousWidth': 400},
                'range': {'category': {'scheme': 'category20'}},  # Default color scheme
                "title": {"fontSize": 18, "font": 'Lato', "anchor": "center", 'color': '#000000', 'fontWeight': 'bold'},
                "legend": {'orient': 'bottom'}
            }
        }
    alt.themes.register('my_theme', my_theme)
    alt.themes.enable('my_theme')

    # Assume df has Month and TTP_NET columns; adjust if needed
    month = df['Month'].max() if not df.empty else 1
    year = pd.to_datetime(cutoff).year  # Use cutoff year

    df_ttp_avg = df[df['Month'] == month].copy()
    df_ttp_avg['Arrow'] = '➟'
    df_ttp_avg['Comment'] = f'{prog} Average {year} = {ttp_avg:.1f}'

    bar_chart = alt.Chart(df).mark_bar(size=45, opacity=0.7).transform_fold(
        fold=['TTP_NET'], as_=['Legend', 'value']
    ).encode(
        x='Month:O',
        y=alt.Y('value:Q', title='Number of Days'),
        color='Legend:N'
    ).properties(width=550, height=250)

    labels = bar_chart.mark_text(
        baseline='middle', dx=3, dy=-10, align='center', fontSize=14, fontWeight=700, color='#000000'
    ).encode(text=alt.Text('value:Q', format=",.2r"))

    time_limit_line = alt.Chart(df_time_limit).mark_line(size=2, color='orange', opacity=0.7).transform_fold(
        fold=['time_limit'], as_=['Legend', 'value']
    ).encode(
        x='Month:O',
        y=alt.Y('value:Q', title=''),
        color='Legend:N'
    ).properties(width=450, height=150)

    ttp_avg_chart = alt.Chart(df_ttp_avg).mark_bar(size=45, color='blue', opacity=0)
    ttp_avg_text = ttp_avg_chart.mark_text(
        baseline='bottom', dy=-70, dx=-60, align='left', fontSize=14, fontWeight=700, color='black'
    ).encode(text=alt.Text('Comment'))
    arrow = ttp_avg_chart.mark_text(dx=-50, dy=-5, angle=90, fontSize=35, color='orange').encode(
        text=alt.Text('Arrow')
    )

    # Total average line in red
    total_avg_line = alt.Chart(pd.DataFrame({'Month': [1, 12], 'Total_Avg': [total_avg] * 2})).mark_line(
        strokeDash=[6, 1], size=4, color='red', opacity=0.7
    ).encode(
        x='Month:O',
        y=alt.Y('Total_Avg:Q', title='Number of Days')
    ).properties(width=450, height=150)

    final_chart = ((bar_chart + labels) + time_limit_line + total_avg_line + (ttp_avg_chart + ttp_avg_text + arrow))
    return final_chart

def generate_ttp_analysis(df_paym, cutoff):
    """
    Generate TTP analysis with tables and charts for each programme and payment type
    """
    # Prepare data
    current_metrics = calculate_current_ttp_metrics(df_paym, cutoff)
    historical_data = load_historical_ttp_data()
    tables = {}
    charts = {}

    # Define payment types and programs
    payment_types = ['IP', 'FP', 'EXPERTS', 'PF']
    programs = ['H2020', 'HEU']

    # Loop by program and payment type
    for prog in programs:
        for pt in payment_types:
            # Generate quarterly table
            table = create_quarterly_ttp_table(df_paym, cutoff, prog, pt)
            tables[f'{prog}_{pt}_table'] = table

            # Prepare data for chart (assuming df_paym has Month and TTP_NET)
            df_chart = df_paym[
                (df_paym['Programme'] == prog) &
                (df_paym['v_payment_type'] == pt)
            ].copy()
            df_chart['Month'] = pd.to_datetime(df_chart['Pay Workflow Last AOS Action Date']).dt.month
            df_chart['TTP_NET'] = pd.to_numeric(df_chart['v_TTP_NET'], errors='coerce')
            df_chart = df_chart[df_chart['TTP_NET'] >= 0]

            # Calculate average for this combination
            avg_ttp_net = current_metrics[prog][pt]['avg_ttp_net'] if pt in current_metrics[prog] else 0
            rolling_avg = df_chart.groupby('Month')['TTP_NET'].mean().reset_index(name='TTP')

            # Generate chart
            chart = chart_machine_ttp(df_chart, pt, prog, avg_ttp_net, rolling_avg)
            charts[f'{prog}_{pt}_chart'] = chart

    return {'tables': tables, 'charts': charts}

In [63]:
# Run the analysis
results = generate_ttp_analysis(df_paym, cutoff)

# Display tables (in Jupyter Notebook)
for key, table in results['tables'].items():
    print(f"\n{key}:")
    display(table)

# Display charts (in Jupyter Notebook)
for key, chart in results['charts'].items():
    print(f"\n{key}:")
    display(chart)

/tmp/ipykernel_128910/670008037.py:128: AltairDeprecationWarning: 
Deprecated since `altair=5.5.0`. Use altair.theme instead.
Most cases require only the following change:

    # Deprecated
    alt.themes.enable('quartz')

    # Updated
    alt.theme.enable('quartz')

If your code registers a theme, make the following change:

    # Deprecated
    def custom_theme():
        return {'height': 400, 'width': 700}
    alt.themes.register('theme_name', custom_theme)
    alt.themes.enable('theme_name')

    # Updated
    @alt.theme.register('theme_name', enable=True)
    def custom_theme():
        return alt.theme.ThemeConfig(
            {'height': 400, 'width': 700}
        )

See the updated User Guide for further details:
    https://altair-viz.github.io/user_guide/api.html#theme
    https://altair-viz.github.io/user_guide/customization.html#chart-themes
  alt.themes.register('my_theme', my_theme)


UnboundLocalError: cannot access local variable 'values' where it is not associated with a value